In [304]:
import pandas as pd
import datetime

In [305]:
shootings = pd.read_csv('https://raw.githubusercontent.com/washingtonpost/data-police-shootings/master/fatal-police-shootings-data.csv')

In [306]:
shootings.shape

(6068, 17)

In [307]:
shootings.isnull().sum()

id                           0
name                       218
date                         0
manner_of_death              0
armed                      208
age                        266
gender                       1
race                       589
city                         0
state                        0
signs_of_mental_illness      0
threat_level                 0
flee                       342
body_camera                  0
longitude                  297
latitude                   297
is_geocoding_exact           0
dtype: int64

In [308]:
# Remove: Longitude, Latitude, Name, ID from dataset
shootings = shootings.drop(columns=['id','name','longitude','latitude'])

In [309]:
# rows that have missing values
miss_value_rows = shootings.loc[shootings.isnull().any(axis=1)]
miss_value_rows.shape

(1163, 13)

In [310]:
# drop that row where gender is not known
shootings = shootings[pd.notnull(shootings['gender'])]

In [317]:
# add age_group column
shootings['age_group'] = pd.cut(shootings['age'], bins=[0, 10, 20, 30,40,50,60,70,80,90,float('Inf')], labels=['0-10', '10-20', '20-30','30-40','40-50','50-60','60-70','70-80','80-90','90-'])


In [312]:
# any other item then 'gun'  or 'unarmed' or 'NaN' is classified as 'other'
# NaNs are transformed to 'unknown'
# levels in armed: gun, other, unarmed, and unknown
shootings['armed'] = shootings['armed'].astype(str)
shootings.loc[shootings['armed'] == 'nan', 'armed'] = 'unknown'
shootings.loc[shootings['armed'].str.contains('gun'), 'armed'] = 'gun'
shootings.loc[(shootings['armed'] != 'gun') & (shootings['armed']!='unknown') & (shootings['armed']!='unarmed'), 'armed'] = 'other'

In [313]:
# do the same thing as above for flee
# levels in flee: flee, no flee, and unknown
shootings['flee'] = shootings['flee'].astype(str)
shootings.loc[shootings['flee'] == 'nan', 'flee'] = 'unknown'
shootings.loc[(shootings['flee'] == 'Car')|(shootings['flee'] == 'Foot')|(shootings['flee'] == 'Other') , 'flee'] = 'flee'
shootings.loc[shootings['flee'] == 'Not fleeing', 'flee'] = 'no flee'

In [314]:
#create Year column from date column
shootings['Year'] = shootings['date'].str.split('-',expand=True)[0]

In [315]:
# Rows that have county instead of cities in their 'city' column
shootings_county = shootings[shootings['city'].str.contains('County')] 

In [316]:
shootings.isnull().sum()

date                         0
manner_of_death              0
armed                        0
age                        266
gender                       0
race                       589
city                         0
state                        0
signs_of_mental_illness      0
threat_level                 0
flee                         0
body_camera                  0
is_geocoding_exact           0
age_group                  266
Year                         0
dtype: int64

In [332]:
shootings.head()

,date,manner_of_death,armed,age,gender,race,city,state,signs_of_mental_illness,threat_level,flee,body_camera,is_geocoding_exact,age_group,Year
0,2015-01-02,shot,gun,53,M,A,Shelton,WA,True,attack,no flee,False,True,50-60,2015
1,2015-01-02,shot,gun,47,M,W,Aloha,OR,False,attack,no flee,False,True,40-50,2015
2,2015-01-03,shot and Tasered,unarmed,23,M,H,Wichita,KS,False,other,no flee,False,True,20-30,2015
3,2015-01-04,shot,other,32,M,W,San Francisco,CA,True,attack,no flee,False,True,30-40,2015
4,2015-01-04,shot,gun,39,M,H,Evans,CO,False,attack,no flee,False,True,30-40,2015


In [334]:
shootings['threat_level'].unique()

array(['attack', 'other', 'undetermined', 'unknown'], dtype=object)

In [336]:
shootings['manner_of_death'].unique()

array(['shot', 'shot and Tasered', 'unknown'], dtype=object)

In [337]:
shootings.to_csv('shootings.csv')